In [18]:
import json
import os
from pprint import pprint
from datetime import datetime, date, timedelta


import tweepy
import requests

headers = {
    'X-API-Key': os.environ["PP_API_KEY"],
}

### get members

In [2]:
member_list = []

with open('../data/member_data.json', 'r') as ofile:
    member_list = json.loads(ofile.read())
ofile.close()

### just get member ids

In [3]:
member_ids = []

for member in member_list:
    member_ids.append(member["id"])

### get most recent bills passed

In [4]:
future_cong = True
this_cong = 123

while future_cong == True:
    resp_json = json.loads(requests.get("https://api.propublica.org/congress/v1/"+ str(this_cong) +"/house/bills/passed.json", headers=headers).text)
    if resp_json["status"] == "500":
        this_cong = this_cong - 1
    else:
        future_cong = False


In [5]:
resp_json

{'status': 'OK',
 'copyright': 'Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.',
 'results': [{'congress': 115,
   'chamber': 'House',
   'num_results': 20,
   'offset': 0,
   'bills': [{'bill_id': 'hres1176-115',
     'bill_slug': 'hres1176',
     'bill_type': 'hres',
     'number': 'H.RES.1176',
     'bill_uri': 'https://api.propublica.org/congress/v1/115/bills/hres1176.json',
     'title': 'Providing for consideration of the conference report to accompany the bill (H.R. 2) to provide for the reform and continuation of agricultural and other programs of the Department of Agriculture through fiscal year 2023, and for other purposes.',
     'short_title': 'Providing for consideration of the conference report to accompany the bill (H.R. 2) to provide for the reform and continuation of agricultural and other programs of the Department of Agriculture through fiscal year 2023, and for other purposes.',
     'sponsor_title': 'Rep.',
     'sponsor_id': 'N000189',
     'sponsor_name

### filter to just this state bills AND that have not been tweeted

In [6]:
this_state_bills = []

tweeted_bill_ids = []

with open('../data/tweeted_bills.json', "r") as ofile:
    tweeted_bill_ids = json.loads(ofile.read())
ofile.close()

for bill in resp_json["results"][0]['bills']:
    if bill["sponsor_id"] in member_ids and bill["bill_id"] not in tweeted_bill_ids:
        this_state_bills.append(bill)

In [7]:
this_state_bills

[{'bill_id': 'hr6687-115',
  'bill_slug': 'hr6687',
  'bill_type': 'hr',
  'number': 'H.R.6687',
  'bill_uri': 'https://api.propublica.org/congress/v1/115/bills/hr6687.json',
  'title': "To direct the Secretary of the Interior to manage the Point Reyes National Seashore in the State of California consistent with Congress' longstanding intent to maintain working dairies and ranches on agricultural property as part of the seashore's unique historic, cultural, scenic and natural values, and for other purposes.",
  'short_title': "To direct the Secretary of the Interior to manage the Point Reyes National Seashore in the State of California consistently with Congress' long-standing intent to continue to authorize working dairies and ranches on agricultural property as part of the se",
  'sponsor_title': 'Rep.',
  'sponsor_id': 'H001068',
  'sponsor_name': 'Jared Huffman',
  'sponsor_state': 'CA',
  'sponsor_party': 'D',
  'sponsor_uri': 'https://api.propublica.org/congress/v1/members/H00106

### craft tweets

In [21]:
congress_conversions = {'115': '115th', '116': '116th', '117': '117th', '118': '118th', '119': '119th', '120': '120th', '121': '121st', '122': '122nd', '123': '123rd'}

max_tweet_len = 280
link_len = 23
allowed_text_length = max_tweet_len - link_len - 5

for bill in this_state_bills:
    member_dict = [x for x in member_list if x["id"] == bill["sponsor_id"]][0]
    
    try:
        full_name = ".@" + member_dict["twitter_id"] + " (" + member_dict["party"] + ")" 
    except:
        full_name = member_dict["role"][0:3] + ". " + member_dict["name"] + " (" + member_dict["party"] + ")" 
    
    house_pass = False
    sen_pass = False
    
    if bill['house_passage'] != None:
        house_pass = True
    if bill['senate_passage'] != None:
        sen_pass = True
        
    
    today_format = datetime.strftime(date.today(), "%Y-%m-%d")
    yesterday_format = datetime.strftime(date.today() - timedelta(1), "%Y-%m-%d")
        
    print(today_format)
    print(yesterday_format)
        
    if today_format not in [bill['house_passage'], bill['senate_passage']] and yesterday_format not in [bill['house_passage'], bill['senate_passage']]:
        continue
    
        
        
        
    t_text = ""
    
    t_text = t_text + bill["number"] + ", introduced by "
    t_text = t_text + full_name + ", "
    
    if house_pass == True and sen_pass == True:
        t_text = t_text + "has passed the House and the Senate."
    elif house_pass == True:
        t_text = t_text + "has passed the House."
    elif sen_pass == True:
        t_text = t_text + "has passed the Senate."
    else:
        continue
        
    
    if len(t_text) > allowed_text_length:
        t_text = t_text[0:allowed_text_length] + "... "
        
        
    congress_ord = congress_conversions[str(this_cong)]
    bill_nums =  [x for x in list(bill["number"]) if x.isdigit()]
    bill_num = "".join(bill_nums)
    bill_chamber = "senate"
    if "H" in bill['number'].upper():
        bill_chamber = "house"
    congress_gov_url = "https://www.congress.gov/bill/"+ congress_ord +"-congress/" + bill_chamber + "-bill/"+ bill_num
    
    t_text = t_text + " " + congress_gov_url
    
#     pprint(bill)
    print(t_text)
    
    bill["t_text"] = t_text

2018-12-13
2018-12-12
2018-12-13
2018-12-12
H.R.6655, introduced by .@RepLaMalfa (R), has passed the House and the Senate. https://www.congress.gov/bill/115th-congress/house-bill/6655
2018-12-13
2018-12-12
2018-12-13
2018-12-12
H.R.6405, introduced by .@RepJeffDenham (R), has passed the House and the Senate. https://www.congress.gov/bill/115th-congress/house-bill/6405


### send the tweets

In [20]:
# send tweets here
auth = tweepy.OAuthHandler(os.environ["T_CONSUMER_KEY"],os.environ["T_CONSUMER_SECRET"])
auth.set_access_token(os.environ["T_ACCESS_TOKEN"],os.environ["T_ACCESS_TOKEN_SECRET"])
api = tweepy.API(auth)
    
    
for bill in this_state_bills:
    print(bill["t_text"])
    print()
    api.update_status(status=bill["t_text"])
    


### log that the bills been tweeted

In [21]:
bill_ids = []

for bill in this_state_bills:
    bill_ids.append(bill["bill_id"]) 
    
tweeted_bill_ids = tweeted_bill_ids + bill_ids

with open('../data/tweeted_bills.json', 'w') as ofile:
    ofile.write(json.dumps(tweeted_bill_ids))
ofile.close()

In [48]:
json.loads(requests.get("https://api.propublica.org/congress/v1/statements/date/2018-10-05.json?offset=200", headers=headers).text)

{'status': 'OK',
 'copyright': 'Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.',
 'num_results': 4,
 'offset': 200,
 'results': [{'url': 'https://www.grassley.senate.gov/news/news-releases/supplemental-fbi-investigation-executive-summary',
   'date': '2018-10-05',
   'title': 'Supplemental FBI Investigation Executive Summary',
   'statement_type': 'Press Release',
   'member_id': 'G000386',
   'congress': 115,
   'member_uri': 'https://api.propublica.org/congress/v1/members/G000386.json',
   'name': 'Charles E. Grassley',
   'chamber': 'Senate',
   'state': 'IA',
   'party': 'R',
   'subjects': []},
  {'url': 'https://www.grassley.senate.gov/news/news-releases/grassley-repeats-request-katz-bromwich-banks-provide-evidence-cited-reported',
   'date': '2018-10-05',
   'title': 'Grassley Repeats Request of Katz, Bromwich, Banks to Provide Evidence Cited, Reported But Never Provided to Committee',
   'statement_type': 'Press Release',
   'member_id': 'G000386',
   'congress': 115,